# Comics Rx
## [A comic book recommendation system](https://github.com/MangrobanGit/comics_rx)
<img src="https://images.unsplash.com/photo-1514329926535-7f6dbfbfb114?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=2850&q=80" width="400" align='left'>

---

# 5 - ALS Model - 'Pseudo' Deployment

This notebook is to explore and develop 'deploying' from a previously saved ALS model.

# Libraries

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2  # 1 would be where you need to specify the files
#%aimport data_fcns

import pandas as pd # dataframes
import os

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

# import necessary libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.sql.types import (StructType, StructField, IntegerType
#                                ,FloatType, LongType, StringType)
from pyspark.sql.types import *

import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, lit, isnan, when, count
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Custom
import data_fcns as dfc
import keys  # Custom keys lib
import comic_recs as cr

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

import time

import numpy as np

In [2]:
# instantiate SparkSession object
spark = pyspark.sql.SparkSession.builder.master("local[*]").getOrCreate()
# spark = SparkSession.builder.master("local").getOrCreate()

## Retrieving Saved Model

In [3]:
comic_rec_model = ALSModel.load('als_filtered')

In [4]:
top_n_df = cr.get_top_n_recs_for_user(spark=spark, model=comic_rec_model, topn=50)
top_n_df

161


,comic_title
1,Criminal (Image)
2,Bitch Planet (Image)
3,Royal City (Image)
4,Black Widow (Marvel)
5,All New Hawkeye (Marvel)
6,Shipwreck (Other)
7,Sex Criminals (Image)
8,Neil Gaiman American Gods Sha (Dark Horse)
9,Spider-Gwen (Marvel)
10,Sweet Tooth (Vertigo)


I'm testing on myself. I'm pretty sure I've bought a few of those title's above. But this could be a failure in how I aggregated on series, but there some evidence of that failing. One example is *Gideon Falls*. There should be only one volume of that. Maybe it's graphic novels? But that shouldn't be an issue (no pun intended) because I believe the original dataset should just be individual comic books. 

Let's test versus the original dataset!

#### Set aside some test series.

- Paper Girls (Image)
- Saga (Other)
- Fade Out (Image)

These I know **for sure** I've bought, if not subscribed.

## Set up connection to AWS RDS